# 뉴스그룹 분류

In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
news_data = fetch_20newsgroups(subset='all', random_state=156)

In [3]:
print(news_data.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
print(news_data.target)

[ 8  8 12 ...  7  3  9]


In [5]:
import pandas as pd
print('target 클래스의 값과 분포도 :')
print(pd.Series(news_data.target).value_counts().sort_index())

target 클래스의 값과 분포도 :
0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64


In [6]:
pd.Series(news_data.target).shape

(18846,)

In [7]:
# subset='train'으로 학습용(Train) 데이터만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', \
                                                        'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target

In [8]:
# subset='test'로 검증용(Test) 데이터만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'),\
                               random_state=156)
X_test = train_news.data
y_test = train_news.target

In [9]:
print('학습{0}, 테스트{1}'.format(len(train_news.data), len(test_news.data)))

학습11314, 테스트7532


## 1.  data에 대한 CountVectorizer 데이터로 transform
## 2. target 값들과 함께 머신러닝 객체로 학습
## 3. 테스트 데이터들에 대한 예측 & 성능 평가

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cnt_vect = CountVectorizer()
# CountVectorizer의 fit : 단어추출 취합 및 인덱싱
cnt_vect.fit(X_train)
# train 데이터에 대한 transform : 빈도수 count 및 좌표 행렬 구성
X_train_cnt_vect = cnt_vect.transform(X_train)
print('학습 데이터 Text의 CountVectorizer Shape : ', X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape :  (11314, 101631)


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# 객체 생성 -> 학습( train_data & target )
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)

C:\Users\204\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
# CountVectorizer의 fit : 단어추출 취합 및 인덱싱
# cnt_vect.fit(X_test) -> 실행되는 순간 취합된 자료가 새로운 자료로 구성되어버림
# 생략 = 기존 피팅 객체로(cnt_vect)로 transform한다는 말입니다

# train 데이터에 대한 transform : 빈도수 count 및 좌표 행렬 구성
X_test_cnt_vect = cnt_vect.transform(X_test)
print('학습 데이터 Text의 CountVectorizer Shape : ', X_test_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape :  (11314, 101631)


__주의__
* 학습데이터에 대해 fit()된 CountVectorizer를 이용해서 테스트 데이터를 피처 벡터화해야 합니다
* 테스트 데이터에서 다시 CountVectorizer의 fit_transform()을 수행하거나 fit()을 수행하면 기존학습데이터의 단어들과 달라진 feature들이 생성되어 기존 학습된 모델에서 가지는 feature의 개수가 달라지며, 머신러닝에 적용할 fit 데이터와 predict 데이터의 형식이 달라지기 때문입니다

In [13]:
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(\
                                                       accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.964


## CounterVectorizer의 업그레이드 버전 TfidVectorizer
* 해당 단어의 한 문서당 출현 빈도수와, 해당 단어의 문서 분포도를 fit와 transform의 구성요소로 포함시킨 모델입니다

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
# 객체 생성
tfidf_vect = TfidfVectorizer()
# X_train으로 fit
tfidf_vect.fit(X_train)
# X_train으로 fit된 객체를 이용하여 X_train을 transform
X_train_tfidf_vect = tfidf_vect.transform(X_train)
# X_train으로 fit된 객체를 이용하여 X_test을 transform
X_test_tfidf_vect = tfidf_vect.transform(X_test)

In [16]:
# LogisticRegression을 이용하여 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_train_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(\
                                                      accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.907


In [17]:
# stopwords 추가
# ngram을 기본(1,1)에서 (1,2)로 변경
# max_df=300 : 300번 이상 출현한 데이터는 추출대상(피쳐)에서 제외
# min_df=10 : 10번 이하 출현한 단어는 추출대상에서 제외
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)

tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

In [ ]:
lf_clf = LogisticRegression()
lf_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도(옵션추가) : {0:.3f}'.format(\
                                                      accuracy_score(y_test, pred)))

## 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합
### GridSearchCV
* 학습과 예측에 영향을 줄 수 있는 파라미터 값들을 여러 값들로 설정하며, 데이터 셋의 분할 또한 횟수만큼 교차 분할하여 여러차례 학습-예측을 반복하는 도구입니다.
* 반복학습 및 예측된 모델 중 가장 성능이 좋았던 모델을 실행결과로 선택하여 대표모델로 재학습시켜 실제 예측에 사용합니다

In [23]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import GridSearchCV
# 하이퍼 파라미터는 머신러닝 모델에 따라 그 종류가 다양하게 사용됩니다
# LogisticRegression의 C 파라미터 : 시그모이드함수()의 그래프에서 곡선의 완만함을 조절할 수 있는 조절값
params = {'C':[5, 10] }
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=2, scoring='accuracy')

In [24]:
# 학습
grid_cv_lr.fit(X_train_tfidf_vect, y_train)

GridSearchCV(cv=2, estimator=LogisticRegression(), param_grid={'C': [5, 10]},
             scoring='accuracy')

In [26]:
# 예측
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('Logistic Regression best C parameter : ', grid_cv_lr.best_params_)
print('TF-IDF Vectorized Logistic Regression(GridSearchCV)의 정확도 : {0:.3f}'\
      .format(accuracy_score(y_test, pred)))

Logistic Regression best C parameter :  {'C': 10}
TF-IDF Vectorized Logistic Regression(GridSearchCV)의 정확도 : 0.973


### Pipeline
* pipeline 함수의 기능을 이용하여 두 개의 동작(Tfidvectorizor의 fit와 transform, LogisticRegression의 fit과 predict를 순서대로 실행
* 파이프라인에 순서대로 들어가 있기 대문에 별도의 Tfidvectorizor 객체의 fit(), transform() 그리고 별도의 LogsticRegression의 fit()과 predict()가 필요없습니다
* pipeline의 fit()과 predict()만으로도 한꺼번에 Feature Vectorization과 머신러닝학습 및 예측이 가능합니다

In [34]:
from sklearn.pipeline import Pipeline
# 파이프라인 구성 및 객체 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), \
                                 max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])
# fit와 predict 함수로 Feature Vectorization과 머신러닝 fit, predict를 순서대로 실행
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline & TF-IDF Vectorized Logistic Regression의 예측 정확도 : {0:.3f}'\
                                            .format(accuracy_score(y_test, pred)))

Pipeline & TF-IDF Vectorized Logistic Regression의 예측 정확도 : 0.973


In [38]:
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])
# pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 GridSearchCV에
# 사용될 파라미터/하이퍼 파라미터 이름과 값을 설정
params = { 'tfidf_vect__ngram_range':[ (1,1), (1,2) ],
           'tfidf_vect__max_df': [100, 300],
           'lr_clf__C': [10]
         }
# 파이프라인 생성 후
# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV( pipeline, param_grid=params, cv=3, \
                                       scoring='accuracy', verbose=1)

In [40]:
grid_cv_pipe.fix(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

AttributeError: 'GridSearchCV' object has no attribute 'fix'

In [ ]:
pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'\
                                 .format(accuracy_score(y_test, pred)))